# Implementation decision tree with ID3 algorithm from scratch

### Author: Sunwoo Choi

### Data
Previously cleaned data
Repository URL: https://github.com/kkehoe1985/ga_data_science_final_project/blob/master/combined_data.csv

Raw data reference: https://raw.githubusercontent.com/kkehoe1985/ga_data_science_final_project/master/combined_data.csv

Thanks to @kkehoe1985 for providing data

In [ ]:
# Set random seed as 1337
import numpy as np

np.random.seed(1337)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SENG 274/Assignment/Assignment1/cleanDf.csv')


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/SENG 274/Assignment/Assignment1/cleanDf.csv')

cleanDf = df.drop(['Votes','UnemploymentRate2015','PovertyAllAgesPct2014','DeepPovAll','AgeTotalPop','MaleRate','FemaleRate','TurnoutRate','Old','Young','Adult',
                   'NomalizePopulation','NomalizeMedHHInc2014','NomalizeToalArea','NomalizePerCapitaInc','NomalizePopDensity','State','County'], axis=1)
# I delete the  'State' and 'County' eventhough it is categorical variables
# This is because this variable makes the tree to overfit.

headers = cleanDf.columns.tolist()

features = headers.copy()
features.remove('Democrat')
print(headers)
print(features)

cleanDf

['Democrat', 'Education', 'Religion', 'EthnicMale', 'EthnicFemale']
['Education', 'Religion', 'EthnicMale', 'EthnicFemale']


,Democrat,Education,Religion,EthnicMale,EthnicFemale
0,0,high school diploma,Other Misc,White,White
1,0,high school diploma,Catholic,White,White
2,0,college,Christian Generic,White,White
3,0,high school diploma,Catholic,White,White
4,0,college,Catholic,White,White
...,...,...,...,...,...
3140,0,college,Catholic,White,White
3141,1,bachelor or higher,Catholic,White,White
3142,0,high school diploma,Mormon,White,White
3143,0,college,Christian Generic,White,White


In [ ]:
# shuffle & split array into training set and validation set
sampleArr = cleanDf.to_numpy()
np.random.shuffle(sampleArr)

splitIdx = (int)(len(sampleArr)*0.7)

trainSet = sampleArr[:splitIdx]
validSet = sampleArr[splitIdx:]

print(len(trainSet)+len(validSet) == len(sampleArr)) # check any missing row

trainDf = pd.DataFrame(trainSet, columns = headers)
validDf = pd.DataFrame(validSet, columns = headers)

True


In [ ]:
validDf

,Democrat,Education,Religion,EthnicMale,EthnicFemale
0,0,high school diploma,Christian Generic,White,White
1,0,high school diploma,Christian Generic,White,White
2,0,high school diploma,Christian Generic,White,White
3,0,bachelor or higher,Christian Generic,White,White
4,0,high school diploma,Christian Generic,White,White
...,...,...,...,...,...
939,0,high school diploma,Christian Generic,White,White
940,0,high school diploma,Christian Generic,White,White
941,0,college,Christian Generic,White,White
942,0,high school diploma,Christian Generic,White,White


In [ ]:
# Calculate entropy
def getEntropy(labelCol):
  entropy = 0
  values = labelCol.value_counts()
  for value in values:
    entropy -= (value/len(labelCol))*np.log(value/len(labelCol))
  return entropy

# calculate conditional entropy
def getCondEntropy(cols, colFeature):
  entropy = 0
  cateFeatures = cols[colFeature].value_counts()
  for cateFeature, count in cateFeatures.items():
    entropy += (count/len(cols[colFeature]))*getEntropy(cols.loc[cols[colFeature] == cateFeature]['Democrat'])
  return entropy

# return attribute which maximize information gain
def selectAttribute(dft, colFeatures):
  if colFeatures is []:
    return 0
  wholeEnt = getEntropy(dft['Democrat'])
  IG = {}
  for colFeature in colFeatures:
    IG[colFeature] = wholeEnt - getCondEntropy(dft[[colFeature,'Democrat']], colFeature)
  return max(IG, key=IG.get)


In [ ]:

def ID3(trainDf, features):
  if len(trainDf['Democrat'].unique()) == 1:
    return {('Label','Democrat') : trainDf['Democrat'].unique()[0]}
  # no more features to split
  if not features:
    vote = trainDf.value_counts()
    return {('Label','Democrat'):vote.idxmax()[0]}
  # all labels are same, classifier is done! 

  
  
  tree = {}
  splitAttribute = selectAttribute(trainDf, features)
  attributes = trainDf[splitAttribute].unique()
  for attribute in attributes:
    tree[(splitAttribute,attribute)] = {}
  for node, subTree in tree.items():
    tmpFeature = features.copy()
    tmpFeature.remove(splitAttribute)

    tree[node] = ID3(trainDf.loc[trainDf[splitAttribute]==node[1]], tmpFeature)
    
  return tree

trainedTree = ID3(trainDf, features)
trainedTree

{('Education',
  'bachelor or higher'): {('Religion',
   'Amish'): {('Label',
    'Democrat'): 0}, ('Religion', 'Catholic'): {('EthnicFemale', 'Black'): {('Label',
     'Democrat'): 1},
   ('EthnicFemale',
    'White'): {('EthnicMale',
     'White'): {('Label', 'Democrat'): 1}}}, ('Religion',
   'Christian Generic'): {('EthnicFemale',
    'Black'): {('Label', 'Democrat'): 1},
   ('EthnicFemale',
    'White'): {('EthnicMale',
     'White'): {('Label', 'Democrat'): 0}}}, ('Religion',
   'Mormon'): {('Label', 'Democrat'): 0}, ('Religion',
   'Other'): {('EthnicMale',
    'White'): {('EthnicFemale', 'White'): {('Label', 'Democrat'): 1}}}},
 ('Education',
  'college'): {('EthnicFemale',
   'Black'): {('Label',
    'Democrat'): 1}, ('EthnicFemale', 'Native'): {('Religion',
    'Catholic'): {('EthnicMale', 'Native'): {('Label', 'Democrat'): 1},
    ('EthnicMale', 'White'): {('Label', 'Democrat'): 0}},
   ('Religion',
    'Christian Generic'): {('Label', 'Democrat'): 0}}, ('EthnicFemale',
   '

In [ ]:
def predict(row,tree):
  for key, item in tree.items():
    category = key[0]
    cateFeature = key[1]
    if category == 'Label':
      return 1 if row['Democrat'] == item else 0
    if row[category] == cateFeature:
      return predict(row, item)
  return 0

def checkAccuracy(checkDf, tree):
  numOfSucess = 0
  for idx, row in checkDf.iterrows():
    numOfSucess += predict(row, tree)
  return numOfSucess/len(checkDf)


validSucRate = checkAccuracy(validDf, trainedTree)
print("Test accuracy with validation data set %f %%" %(validSucRate*100))



Test accuracy with validation data set 90.466102 %


In [ ]:
# Bonus part 1

newDf = df.copy()

genderRate = []
youngOrOld = []
for idx, row in df.iterrows():
  if row['Young'] < row['Old']:
    youngOrOld.append('Old')
  else:
    youngOrOld.append('Young')
  if row['MaleRate'] > row['FemaleRate']:
    genderRate.append('Male')
  else:
    genderRate.append('Female')

newDf['GenderRate'] = genderRate
newDf['YoungOrOld'] = youngOrOld

newDf.drop(['Votes','UnemploymentRate2015','PovertyAllAgesPct2014','DeepPovAll','AgeTotalPop','MaleRate','FemaleRate','TurnoutRate','Old','Young','Adult',
                   'NomalizePopulation','NomalizeMedHHInc2014','NomalizeToalArea','NomalizePerCapitaInc','NomalizePopDensity','State','County'], axis=1, inplace=True)

headers = newDf.columns.tolist()

features = headers.copy()
features.remove('Democrat')

# shuffle & split array into training set and validation set
sampleArr = newDf.to_numpy()
np.random.shuffle(sampleArr)

splitIdx = (int)(len(sampleArr)*0.7)

trainSet = sampleArr[:splitIdx]
validSet = sampleArr[splitIdx:]

trainDf = pd.DataFrame(trainSet, columns = headers)
validDf = pd.DataFrame(validSet, columns = headers)

newDf

,Democrat,Education,Religion,EthnicMale,EthnicFemale,GenderRate,YoungOrOld
0,0,high school diploma,Other Misc,White,White,Female,Young
1,0,high school diploma,Catholic,White,White,Female,Young
2,0,college,Christian Generic,White,White,Female,Young
3,0,high school diploma,Catholic,White,White,Female,Young
4,0,college,Catholic,White,White,Female,Young
...,...,...,...,...,...,...,...
3140,0,college,Catholic,White,White,Male,Young
3141,1,bachelor or higher,Catholic,White,White,Male,Young
3142,0,high school diploma,Mormon,White,White,Male,Young
3143,0,college,Christian Generic,White,White,Male,Young


In [ ]:
trainedTree = ID3(trainDf, features)
validSucRate = checkAccuracy(validDf, trainedTree)
print("Test accuracy with validation data set %f %%" %(validSucRate*100))


Test accuracy with validation data set 91.843220 %


The accuracy increase 1.5% by adding two categrocal features which are `YoungOrOld` and `GenderRate`. 

In [ ]:
# Bonus part : Gini 
def getGini(labelCol):
  gini = 1
  values = labelCol.value_counts()
  for value in values:
    gini -= ((value/len(labelCol)))**2
  return gini

# calculate conditional entropy
def getGiniIndex(cols, colFeature):
  prob = 0
  cateFeatures = cols[colFeature].value_counts()
  for cateFeature, count in cateFeatures.items():
    prob += (count/len(cols[colFeature]))*getGini(cols.loc[cols[colFeature] == cateFeature]['Democrat'])
  return prob

# return attribute which maximize information gain
def selectAttributeGini(dft, colFeatures):
  if colFeatures is []:
    return 0
  wholeGini = getGini(dft['Democrat'])
  allGini = {}
  for colFeature in colFeatures:
    allGini[colFeature] = wholeGini - getGiniIndex(dft[[colFeature,'Democrat']], colFeature)
  return max(allGini, key=allGini.get)


In [ ]:
def CART(trainDf, features):
  if len(trainDf['Democrat'].unique()) == 1:
    return {('Label','Democrat') : trainDf['Democrat'].unique()[0]}
  # no more features to split
  if not features:
    vote = trainDf.value_counts()
    return {('Label','Democrat'):vote.idxmax()[0]}
  # all labels are same, classifier is done! 
  
  tree = {}
  splitAttribute = selectAttributeGini(trainDf, features)
  attributes = trainDf[splitAttribute].unique()
  for attribute in attributes:
    tree[(splitAttribute,attribute)] = {}
  for node, subTree in tree.items():
    tmpFeature = features.copy()
    tmpFeature.remove(splitAttribute)

    tree[node] = CART(trainDf.loc[trainDf[splitAttribute]==node[1]], tmpFeature)
    
  return tree

In [ ]:
features = headers.copy()
features.remove('Democrat')
trainedTree = CART(trainDf, features)
validSucRate = checkAccuracy(validDf, trainedTree)
print("Test accuracy with validation data set %f %%" %(validSucRate*100))

Test accuracy with validation data set 91.843220 %


The accuracys of ID3 algorithm and CART algorithm are same.

I think it is becuase we have small number of features and the spliting decision by Gini and Entropy are same.
